#Working With Text Data

In [1]:
from sklearn.datasets import fetch_20newsgroups

##A fim de obter tempos de execução mais rápidos para este primeiro exemplo, trabalharemos em um conjunto de dados parcial com apenas 4 categorias das 20 disponíveis no conjunto de dados:

In [2]:
categories = ['alt.atheism', 'soc.religion.christian',
              'comp.graphics', 'sci.med']

##Agora podemos carregar a lista de arquivos que correspondem a essas categorias da seguinte maneira:

In [3]:
twenty_train = fetch_20newsgroups(subset='train',
     categories=categories, shuffle=True, random_state=42)

In [4]:
type(twenty_train)

sklearn.utils.Bunch

##O conjunto de dados retornado é um "grupo" (bunch) de scikit-learn: um objeto de suporte simples com campos que podem ser acessados ​​como chaves python dict ou atributos de objeto por conveniência, por exemplo, o target_names contém a lista dos nomes de categoria solicitados:

In [5]:
twenty_train.target_names

['alt.atheism', 'comp.graphics', 'sci.med', 'soc.religion.christian']

##Os próprios arquivos são carregados na memória no atributo "data". Para referência, os nomes dos arquivos também estão disponíveis

In [7]:
len(twenty_train.data)
len(twenty_train.filenames)

2257

##Vamos imprimir as primeiras linhas do primeiro arquivo carregado:

In [8]:
print("\n".join(twenty_train.data[0].split("\n")[:3]))

From: sd345@city.ac.uk (Michael Collier)
Subject: Converting images to HP LaserJet III?
Nntp-Posting-Host: hampton


In [9]:
print(twenty_train.target_names[twenty_train.target[0]])

comp.graphics


##Algoritmos de aprendizado supervisionado exigirão um rótulo de categoria para cada documento no conjunto de treinamento. Nesse caso, a categoria é o nome do grupo de notícias que também é o nome da pasta que contém os documentos individuais. Por motivos de velocidade e eficiência de espaço, o scikit-learn carrega o atributo target como uma matriz de inteiros que corresponde ao índice do nome da categoria na lista target_names. O ID de categoria inteira de cada amostra é armazenado no atributo de destino:

In [10]:
twenty_train.target[:10]

array([1, 1, 3, 3, 3, 3, 3, 2, 2, 2], dtype=int64)

##É possível recuperar os nomes das categorias da seguinte forma:

In [11]:
for t in twenty_train.target[:10]:
     print(twenty_train.target_names[t])

comp.graphics
comp.graphics
soc.religion.christian
soc.religion.christian
soc.religion.christian
soc.religion.christian
soc.religion.christian
sci.med
sci.med
sci.med


##Você deve ter notado que as amostras foram embaralhadas aleatoriamente quando chamamos fetch_20newsgroups (..., shuffle = True, random_state = 42): isso é útil se você deseja selecionar apenas um subconjunto de amostras para treinar rapidamente um modelo e obter um primeiro ideia dos resultados antes de treinar novamente no conjunto de dados completo mais tarde

#Extracting features from text files

##Para realizar o aprendizado de máquina em documentos de texto, primeiro precisamos transformar o conteúdo do texto em vetores de recursos numéricos.

##Sacos de palavras

A maneira mais intuitiva de fazer isso é usar uma representação de sacos de palavras: Atribua um ID de número inteiro fixo para cada palavra que ocorre em qualquer documento do conjunto de treinamento (por exemplo, construindo um dicionário de palavras para índices inteiros). Para cada documento #i, conte o número de ocorrências de cada palavra w e armazene-o em X [i, j] como o valor da característica #j onde j é o índice da palavra w no dicionário. A representação dos pacotes de palavras implica que n_features é o número de palavras distintas no corpus: esse número é normalmente maior do que 100.000. Se n_samples == 10000, o armazenamento de X como uma matriz NumPy do tipo float32 exigiria 10000 x 100000 x 4 bytes = 4 GB em RAM, o que dificilmente é gerenciável nos computadores de hoje. Felizmente, a maioria dos valores em X serão zeros, pois, para um determinado documento, serão usados ​​menos de alguns milhares de palavras distintas. Por esse motivo, dizemos que pacotes de palavras são tipicamente conjuntos de dados esparsos de alta dimensão. Podemos economizar muita memória armazenando apenas as partes diferentes de zero dos vetores de recursos na memória. As matrizes scipy.sparse são estruturas de dados que fazem exatamente isso, e o scikit-learn tem suporte integrado para essas estruturas.

##Tokenização de texto

Com scikit-learn Pré-processamento de texto, tokenização e filtragem de palavras irrelevantes estão todos incluídos no CountVectorizer, que cria um dicionário de recursos e transforma documentos em vetores de recursos:

In [12]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(twenty_train.data)
X_train_counts.shape

(2257, 35788)

##CountVectorizer suporta contagens de N-gramas de palavras ou caracteres consecutivos. Uma vez ajustado, o vetorizador construiu um dicionário de índices de recursos:

In [13]:
count_vect.vocabulary_.get(u'algorithm')

4690

##O valor do índice de uma palavra no vocabulário está relacionado à sua frequência em todo o corpus de treinamento

##De ocorrências a frequências

A contagem de ocorrências é um bom começo, mas há um problema: documentos mais longos terão valores médios de contagem mais altos do que documentos mais curtos, embora possam falar sobre os mesmos tópicos. Para evitar essas discrepâncias em potencial, é suficiente dividir o número de ocorrências de cada palavra em um documento pelo número total de palavras no documento: esses novos recursos são chamados de tf para Frequências de termo. Outro refinamento além do tf é reduzir os pesos das palavras que ocorrem em muitos documentos no corpus e, portanto, são menos informativas do que aquelas que ocorrem apenas em uma parte menor do corpus. Essa redução é chamada de tf – idf para “Frequência do termo vezes frequência inversa do documento”. Tanto tf quanto tf – idf podem ser calculados da seguinte maneira usando TfidfTransformer:

In [14]:
from sklearn.feature_extraction.text import TfidfTransformer
tf_transformer = TfidfTransformer(use_idf=False).fit(X_train_counts)
X_train_tf = tf_transformer.transform(X_train_counts)
X_train_tf.shape

(2257, 35788)

##No código de exemplo acima, primeiro usamos o método fit (..) para ajustar nosso estimador aos dados e, em segundo lugar, o método transform (..) para transformar nossa matriz de contagem em uma representação tf-idf. Essas duas etapas podem ser combinadas para atingir o mesmo resultado final mais rápido, ignorando o processamento redundante. Isso é feito usando o método fit_transform (..) conforme mostrado abaixo e conforme mencionado na nota da seção anterior:

In [15]:
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

(2257, 35788)

#Treinando um classificador 

Agora que temos nossos recursos, podemos treinar um classificador para tentar prever a categoria de uma postagem. Vamos começar com um classificador Bayes ingênuo, que fornece uma boa base para esta tarefa. O scikit-learn inclui várias variantes desse classificador; o mais adequado para contagem de palavras é a variante multinomial:

In [16]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train_tfidf, twenty_train.target)

##Para tentar prever o resultado em um novo documento, precisamos extrair os recursos usando quase a mesma cadeia de extração de recursos de antes. A diferença é que chamamos transform em vez de fit_transform nos transformadores, uma vez que eles já foram ajustados ao conjunto de treinamento:

In [17]:
docs_new = ['God is love', 'OpenGL on the GPU is fast']
X_new_counts = count_vect.transform(docs_new)
X_new_tfidf = tfidf_transformer.transform(X_new_counts)

predicted = clf.predict(X_new_tfidf)

for doc, category in zip(docs_new, predicted):
    print('%r => %s' % (doc, twenty_train.target_names[category]))

'God is love' => soc.religion.christian
'OpenGL on the GPU is fast' => comp.graphics


#Building a pipeline

In order to make the vectorizer => transformer => classifier easier to work with, scikit-learn provides a Pipeline class that behaves like a compound classifier:

In [18]:
from sklearn.pipeline import Pipeline
text_clf = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', MultinomialNB()),
])

##Os nomes vect, tfidf e clf (classificador) são arbitrários. Iremos usá-los para realizar o grid search para hiperparâmetros adequados abaixo. Agora podemos treinar o modelo com um único comando:

In [19]:
text_clf.fit(twenty_train.data, twenty_train.target)

Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('clf', MultinomialNB())])

#Avaliação do desempenho no conjunto de teste

Avaliar a precisão preditiva do modelo é igualmente fácil:

In [20]:
import numpy as np
twenty_test = fetch_20newsgroups(subset='test',
    categories=categories, shuffle=True, random_state=42)
docs_test = twenty_test.data
predicted = text_clf.predict(docs_test)
np.mean(predicted == twenty_test.target)

0.8348868175765646

##Atingimos 83,5% de precisão. Vamos ver se podemos fazer melhor com uma máquina de vetores de suporte linear (SVM), que é amplamente considerada como um dos melhores algoritmos de classificação de texto (embora também seja um pouco mais lenta do que o ingênuo Bayes). Podemos mudar o aluno simplesmente conectando um objeto classificador diferente em nosso pipeline: